# Homework 2

The goal of this homework is to get familiar with MLflow.

In [1]:
!python --version

Python 3.10.4


## Q1. Install MLFlow

To get started with MLflow you'll need to install the MLflow Python package.

For this we recommend creating a separate Python environment, for example, you can use [conda environments](https://docs.conda.io/projects/conda/en/latest/user-guide/getting-started.html#managing-envs), and then install the package there with `pip` or `conda`.

Once you installed the package, run the command `mlflow --version` and check the output.

What's the version that you have?

In [1]:
import os
import shutil
import pathlib
import warnings
warnings.filterwarnings("ignore")

In [2]:
try:
    import mlflow
except:
    %pip install --user mlflow
    import mlflow
print(mlflow.__version__)

1.26.0


Setup mlflow experiment

In [3]:
mlflow_root_dir = pathlib.Path(f"{os.getcwd()}/.mlflow/")
mlflow_tracking_path = f"sqlite:///{mlflow_root_dir / 'db.sqlite'}"
mlflow_artifact_path = mlflow_root_dir / "artifacts/"
mlflow_experiment_name = "homework-02"

try:
    shutil.rmtree(mlflow_root_dir)
except:
    pass
pathlib.Path(mlflow_root_dir).mkdir(parents=True, exist_ok=True)
os.environ["MLFLOW_TRACKING_URI"] = mlflow_tracking_path

mlflow_client = mlflow.tracking.MlflowClient(
    tracking_uri=mlflow_tracking_path,
)
experiment_id = mlflow_client.create_experiment(
    name=mlflow_experiment_name,
    artifact_location=str(mlflow_artifact_path),
)
print(f"Created {experiment_id = }")

2022/05/26 13:22:30 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2022/05/26 13:22:30 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

Created experiment_id = '1'


## Q2. Download and preprocess the data

We'll use the same dataset used in the videos of the 1st and 2nd module: "Green Taxi Trip Records".

Download the data for January, February and March 2021 in parquet format from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

Use the script `preprocess_data.py` located in the folder `homework` to preprocess the data.

The script will

- load the data from the folder `<TAXI_DATA_FOLDER>` (the folder where you have downloaded the data),
- fit a `DictVectorizer` on the training set (January 2021 data),
- save the preprocessed datasets and the model to disk

Your task is to download the datasets and then execute this command:

```
python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
```

Tip: go to `02-experiment-tracking/homework/` folder before executing the command and change the value of `<TAXI_DATA_FOLDER>` to the location where you saved the data.

How many files were saved to `OUTPUT_FOLDER`?

* 1
* 3
* 4
* 7


Jupyter kernel is already launched in `02-experiment-tracking/` dir:

In [4]:
!ls

README.md                    meta.json
duration-prediction.ipynb    mlruns
homework                     model-registry.ipynb
homework.md                  output
homework_olegtaratuhin.ipynb requirements.txt
images                       running-mlflow-examples


In [5]:
%%bash
    cd homework && \
    python preprocess_data.py --raw_data_path ../../data --dest_path ../output && \
    ls ../output && \
    ls -1q ../output/* | wc -l

dv.pkl
test.pkl
train.pkl
valid.pkl
       4


How many files where saved?

## Q3. Train a model with autolog

We will train a random forest regressor on the taxi dataset.

We have prepared the training script `train.py` for this exercise, which can be also found in the folder `homework`. The script will load the datasets produced by the previous step, train the model on the training set and finally calculate the RMSE on the validation set.

Your task is to modify the script to enable **autologging** with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked. 

> Tip 1: don't forget to wrap the training code with a `with mlflow.start_run():` statement as we showed in the videos.

> Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

How many parameters are automatically logged by MLflow?

* 19
* 17
* 10
* 20


Modify preprocessing file to enable autologging:

In [6]:
!cd homework && diff train.py train_mlflow.py

1a2
> import functools
3a5
> from typing import Any, Callable
6a9
> import mlflow
13a17,28
> def mlflow_autolog(func: Callable[..., Any]) -> Callable[..., Any]:
>     @functools.wraps(func)
>     def inner(*args, **kwargs) -> Any:
>         mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URI"))
>         mlflow.set_experiment("homework-02")
>         mlflow.set_tag("dev", "olegtaratuhin")
>         mlflow.autolog()
>         return func(*args, **kwargs)
>     return inner
> 
> 
> @mlflow_autolog


In [7]:
!cd homework && python train_mlflow.py --data_path ../output

2022/05/26 13:22:52 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2022/05/26 13:23:03 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/oleg.taratukhin/miniforge3/envs/mlops/lib/python3.10/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."


In [22]:
experiment = mlflow_client.get_experiment_by_name(mlflow_experiment_name)
runs = mlflow_client.search_runs(experiment_id, order_by=["attribute.start_time DESC"])
latest_run = runs[0]
print(f"Autologging produced {len(latest_run.data.params)} parameters")

Autologging produced 17 parameters


## Q4. Launch the tracking server locally

We want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry.

Your task is to

* launch the tracking server on your local machine
* select a SQLite db for the backend store and a folder called `artifacts` for the artifacts store

You should keep the tracking server running to work on the next two exercises that use the server.

In addition to `backend-store-uri`, what else do you need to pass to properly configure the server?

* `default-artifact-root`
* `serve-artifacts`
* `artifacts-only`
* `artifacts-destination`

In [ ]:
!mlflow server --backend-store-uri sqlite:///db.sqlite --default-artifact-root ./runs/ &

## Q5. Tune the hyperparameters of the model

Now let's try to reduce the validation error by tuning the hyperparameters of the random forest regressor using `hyperopt`.

We have prepared the script `hpo.py` for this exercise. 

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to MLflow for each run of the hyperparameter optimization (you will need to add a few lines of code to the `objective` function) and run the script without passing any parameters.

After that, open the MLflow UI and explore the runs from the experiment called `random-forest-hyperopt` to answer the question below.

Important: don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

* the list of hyperparameters that are passed to the `objective` function during the optimization.
* the RMSE obtained on the validation set (February 2021 data).

What's the best validation RMSE that you got?

* 6.128
* 6.628
* 7.128
* 7.628

In [23]:
!cd homework && diff hpo.py hpo_mlflow.py

22,24c22,25
< 
<     X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
<     X_valid, y_valid = load_pickle(os.path.join(data_path, "valid.pkl"))
---
>     train_data_path = os.path.join(data_path, "train.pkl")
>     valid_data_path = os.path.join(data_path, "valid.pkl")
>     X_train, y_train = load_pickle(train_data_path)
>     X_valid, y_valid = load_pickle(valid_data_path)
27,31c28,39
< 
<         rf = RandomForestRegressor(**params)
<         rf.fit(X_train, y_train)
<         y_pred = rf.predict(X_valid)
<         rmse = mean_squared_error(y_valid, y_pred, squared=False)
---
>         with mlflow.start_run():
>             mlflow.log_param("train_data_path", train_data_path)
>             mlflow.log_param("valid_data_path", valid_data_path)
>             mlflow.set_tag("dev", "olegtaratukhin")
> 
>             mlflow.log_params(params)
>             rf = RandomForestRegressor(**params)
>     
>             rf.fit(X_train, y_train)
>             y_pred = rf.pred

In [25]:
!python homework/hpo_mlflow.py --data_path output

100%|█████████| 50/50 [03:03<00:00,  3.66s/trial, best loss: 6.6284257482044735]


In [32]:
rf_opt_experiment = mlflow_client.get_experiment_by_name("random-forest-hyperopt")
rf_opt_experiment_runs = mlflow_client.search_runs(
    experiment_ids=rf_opt_experiment.experiment_id,
    order_by=["metrics.rmse ASC"],
)
rf_opt_experiment_best_metrics = rf_opt_experiment_runs[0].data.metrics
print(f"Best metric value: {rf_opt_experiment_best_metrics['rmse']}")

Best metric value: 6.6284257482044735


In [47]:
rf_opt_experiment_runs[0].info.run_id

'2790d3c34c584928bfce46d43d484184'

## Q6. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good so we can assume that we are ready to test some of these models in production. In this exercise, you'll promote the best model to the model registry. We have prepared a script called `register_model.py`, which will check the results from the previous step and select the top 5 runs. After that, it will calculate the RMSE of those models on the test set (March 2021 data) and save the results to a new experiment called `random-forest-best-models`.

Your task is to update the script `register_model.py` so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

> Tip 1: you can use the method `search_runs` from the `MlflowClient` to get the model with the lowest RMSE.

> Tip 2: to register the model you can use the method `mlflow.register_model` and you will need to pass the right model_uri in the form of a string that looks like this: `"runs:/<RUN_ID>/model"`, and the name of the model (make sure to choose a good one!).

What is the test RMSE of the best model?

* 6.1
* 6.55
* 7.93
* 15.1

In [62]:
!cd homework && diff register_model.py register_model_final.py

68c68,71
<     # best_run = client.search_runs( ...  )[0]
---
>     best_run = client.search_runs(
>         experiment_ids=experiment.experiment_id,
>         order_by=["metrics.rmse ASC"]
>     )[0]
71c74,77
<     # mlflow.register_model( ... )
---
>     mlflow.register_model(
>         model_uri=f"runs:/{best_run.info.run_id}/model",
>         name="random-forrest-model",
>     )
84a91
>         type=int,


In [65]:
!python homework/register_model_final.py --data_path output --top_n 5

2022/05/26 17:26:30 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/oleg.taratukhin/miniforge3/envs/mlops/lib/python3.10/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."
Successfully registered model 'random-forrest-model'.
2022/05/26 17:27:03 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: random-forrest-model, version 1
Created version '1' of model 'random-forrest-model'.


In [68]:
rf_opt_experiment = mlflow_client.get_experiment_by_name("random-forest-best-models")
rf_opt_experiment_runs = mlflow_client.search_runs(
    experiment_ids=rf_opt_experiment.experiment_id,
    order_by=["metrics.test_rmse ASC"],
)
rf_opt_experiment_best_metrics = rf_opt_experiment_runs[0].data.metrics
print(f"Best metric value: {rf_opt_experiment_best_metrics['test_rmse']}")

Best metric value: 6.547886185595423
